In [61]:
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get('https://us.shein.com/')
WebDriverWait(driver, timeout=10)

#click out of coupon pop up
coupon_box = driver.find_elements(By.CLASS_NAME, "c-coupon-box")
if (coupon_box):
  print("coupon")
  WebDriverWait(driver, timeout=2)
  x_box = driver.find_element(By.CLASS_NAME, "iconfont icon-close she-close")
  x_box.click()

coupon


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".iconfont icon-close she-close"}
  (Session info: chrome=118.0.5993.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001004d0510 chromedriver + 4310288
1   chromedriver                        0x00000001004c84bc chromedriver + 4277436
2   chromedriver                        0x00000001000fbb6c chromedriver + 293740
3   chromedriver                        0x0000000100141040 chromedriver + 577600
4   chromedriver                        0x000000010017be60 chromedriver + 818784
5   chromedriver                        0x0000000100134fd0 chromedriver + 528336
6   chromedriver                        0x0000000100135e7c chromedriver + 532092
7   chromedriver                        0x0000000100496834 chromedriver + 4073524
8   c

In [60]:
# get search box
search_box = driver.find_element(By.CLASS_NAME, "header-search-input j-header-search-input")
search_box.click()
#type in keyword
# get search buttom
search_button = driver.find_element(By.CLASS_NAME, "search-btn she-btn-black j-search-btn")
search_button.click()
# price
#price = driver.find_element(By.CLASS_NAME, "product-item__camelcase-wrap")X

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".header-search-input j-header-search-input"}
  (Session info: chrome=118.0.5993.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001004d0510 chromedriver + 4310288
1   chromedriver                        0x00000001004c84bc chromedriver + 4277436
2   chromedriver                        0x00000001000fbb6c chromedriver + 293740
3   chromedriver                        0x0000000100141040 chromedriver + 577600
4   chromedriver                        0x000000010017be60 chromedriver + 818784
5   chromedriver                        0x0000000100134fd0 chromedriver + 528336
6   chromedriver                        0x0000000100135e7c chromedriver + 532092
7   chromedriver                        0x0000000100496834 chromedriver + 4